In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
sample=pd.read_csv("/content/drive/MyDrive/Contest ML/obesity-level-prediction-fall-2024/sample_submission.csv")

test=pd.read_csv("/content/drive/MyDrive/Contest ML/obesity-level-prediction-fall-2024/test.csv")
print(test.head(10))
#show Data set
print(sample.head(10))
train=pd.read_csv("/content/drive/MyDrive/Contest ML/obesity-level-prediction-fall-2024/train.csv")

print(train.head(10))

In [ ]:

print("Null values in the training dataset:\n", train.isnull().sum())

print("\nNull values in the testing dataset:\n", test.isnull().sum())

In [ ]:

print("\nData types of the training dataset:\n", train.dtypes)
print("\nData types of the testing dataset:\n", test.dtypes)

In [8]:

numerical_cols = train.select_dtypes(include=['float64', 'int64']).columns

for col in numerical_cols:
    # Handle cases where all values might be NaN in a column
    if train[col].isnull().all():
        # If all values are NaN use 0 imputation, this is problem specific.
        train[col] = 0
    else:
        train[col] = train[col].fillna(train[col].median())


In [9]:

categorical_cols = train.select_dtypes(include=['object']).columns

for col in categorical_cols:
    train[col] = train.groupby('NObeyesdad')[col].transform(lambda x: x.fillna(x.mode()[0]))

In [ ]:
train.isnull().sum()
#test.isnull().sum()

In [ ]:

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Initialize LabelEncoder
le = LabelEncoder()

# Identify categorical columns (excluding ID and target variable)
categorical_cols = train.select_dtypes(include=['object']).columns
categorical_cols = [col for col in categorical_cols if col not in ['ID', 'NObeyesdad']]

# Apply Label Encoding to categorical features
for col in categorical_cols:
    train[col] = le.fit_transform(train[col])

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), categorical_cols)], remainder='passthrough')

train_encoded = ct.fit_transform(train)


train.dtypes

In [14]:

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier  # Example model, you can change this
from sklearn.metrics import accuracy_score


X = train.drop(columns=['NObeyesdad', 'ID'], axis=1)
y = train['NObeyesdad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define the model (RandomForestClassifier as an example, you can use any model)
model = RandomForestClassifier(random_state=42)


param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],    
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4]     
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1) # Use all available cores
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print(f"Best Hyperparameters: {best_params}")


# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the best model on the test set: {accuracy}")


Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy of the best model on the test set: 0.7927927927927928


In [29]:

from sklearn.model_selection import train_test_split
X=train.drop(columns=['NObeyesdad'], axis=1)
y=train['NObeyesdad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8558558558558559


In [30]:
rf=RandomForestClassifier(random_state=42)
rf.fit(X_train,y_train)
rf.score(X_test, y_test)

0.8558558558558559

In [31]:

numerical_cols1 = test.select_dtypes(include=['float64', 'int64']).columns
test[numerical_cols1] = test[numerical_cols].fillna(test[numerical_cols].mean())

In [ ]:
categorical_cols1 = test.select_dtypes(include=['object']).columns
test[categorical_cols1] = test[categorical_cols1].fillna(test[categorical_cols1].mode().iloc[0])

test.isnull().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le1=LabelEncoder()
ctg1=test.drop(columns=['ID', 'Age', 'Height', 'Weight', 'NCP', 'CH2O', 'FAF', 'TUE', 'FCVC', 'NCP'])
for col in ctg1.columns:
  test[col]=le1.fit_transform(test[col])
test.dtypes

In [41]:

pred = rf.predict(test)
pred

array(['Overweight_Level_II', 'Overweight_Level_II',
       'Overweight_Level_II', ..., 'Overweight_Level_II',
       'Overweight_Level_II', 'Overweight_Level_II'], dtype=object)

In [45]:
submission22=pd.DataFrame({'ID':test['ID'], 'NObeyesdad':pred})
submission22.to_csv('submission223.csv', index=False)
df=pd.read_csv('submission223.csv')